In [0]:
#Downloading data and unzipping files from machine hack website
import requests, zipfile, io
zip_file_url = "https://www.machinehack.com/wp-content/uploads/2020/03/MovieScriptsParticipantsData.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [2]:
pip install transformers
#Colab doesnt have transformers library preinstalled.

     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 14.1MB/s 
     |████████████████████████████████| 890kB 19.2MB/s 
     |████████████████████████████████| 3.8MB 26.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=fa84331caa899d2a1f0f9e23b258990fde7e242a66982fdea4abab161148f531
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings;
warnings.filterwarnings('ignore');
pd.set_option('display.max_colwidth', 100)
import torch
import time
import pickle
import os
from transformers import DistilBertTokenizer,DistilBertModel
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss,accuracy_score
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


cuda


In [0]:
seed = 123
#Fixing seed to ensure reproducibility
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

In [0]:
cur_dir = os.getcwd()
input_dir = '/content/MovieScriptsParticipantsData'
script_dir = '/content/MovieScriptsParticipantsData/Scripts'

In [0]:
train_path = os.path.join(cur_dir,input_dir,'Train.csv')
test_path = os.path.join(cur_dir,input_dir,'Test.csv')

In [0]:
df_raw = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1978 entries, 0 to 1977
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   File_Name  1978 non-null   object
 1   Labels     1978 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.0+ KB


In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   File_Name  849 non-null    object
dtypes: object(1)
memory usage: 6.8+ KB


In [11]:
df_test['Labels'] = -1 
df_tot = pd.concat([df_raw,df_test], axis = 0)
#Concatenatig both train and test sets for easy processing
df_tot.reset_index(inplace= True,drop=True)
len(df_tot)

2827

In [12]:
df_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2827 entries, 0 to 2826
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   File_Name  2827 non-null   object
 1   Labels     2827 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 44.3+ KB


In [13]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
# Initializing preinitialized distilbert tokenizer and model. Distilbert chosen based on memory and speed criteria.

In [0]:
#Reading scripts from the files
def script_reader(file_name):
    script_path = os.path.join(cur_dir,script_dir,file_name)
    with open (script_path, "r") as myfile:
        script =  myfile.read()
        return script

In [0]:
#Each script at a time is converted into a datset and processed
class ScriptDataset(Dataset):
    def __init__(self, file_path,tokenizer,seq_len = 510):
        #Sequence length = Maximum possible sequence length(512) which can be fed to model - start and end tokens(2) 
        self.file_path =  file_path
        self.script = script_reader(self.file_path)
        #Each script is loaded only when it is being processed and deleted once tokenization and encoding is done
        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.tokens = self.tokenizer.tokenize(self.script)
        del self.script
        self.sequences = [self.tokens[i:i+self.seq_len] for i in range(0,len(self.tokens),self.seq_len)][:-1]
        #Drops last sequence of unequal length
        del self.tokens
        
    def __len__(self):
        return len(self.sequences) 
    
    def __getitem__(self, idx):
        return torch.tensor(self.tokenizer.encode(self.sequences[idx], add_special_tokens=True))

In [0]:
#Dividing a script into sequence and creating a embedding for each sequence and concatening all those sequence embeddings
def create_bert_embedding(file_path,tokenizer,model,batch_size = 2):
    model = model.to(device)
    model = model.eval()
    
    script_set = ScriptDataset(file_path,tokenizer)
    batch_loader = DataLoader(script_set,batch_size = batch_size, shuffle = False,drop_last=False)
    del script_set
    
    seq_embeddings = []
    for i,batch in enumerate(batch_loader):
        batch = batch.to(device)
        #Avoid gradient calculation , for speed and memory issues
        with torch.no_grad():
            seq_embed = model(batch)[0][:,0,:]
            #Collecting only the final hidden state corresponding to CLS token which can be used as sequence embedding(imperfect but still works)
        del batch
        seq_embeddings.append(seq_embed)
        del seq_embed
    del batch_loader
           
    return torch.cat(seq_embeddings,dim = 0).cpu().detach().numpy()

In [0]:
tik = time.time()
Embeddings = []
for i in range(len(df_tot)):
    Embeddings.append(create_bert_embedding(df_tot.File_Name[i],tokenizer,model))
    if i % 50 == 0:
        print(i)

print('Embedding completed')

with open('/content/drive/My Drive/Script_Classification/embeddings.pickle', 'wb') as handle:
    pickle.dump(Embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Saving the embeddings in the drive incase the colab session gets terminated 
print('Pickling completed')


df_tot['Embeds'] = Embeddings
tok = time.time()
print (f'{round(tok-tik,2)}s')

with open('/content/drive/My Drive/Script_Classification/df_total.pickle', 'wb') as handle:
    pickle.dump(df_tot, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Saving the dataframe so that further analysis can start from this point

In [0]:
df_tot_path = "/content/drive/My Drive/Script_Classification/df_total.pickle"

with open(df_tot_path, 'rb') as handle:
    df_tot = pickle.load(handle)

In [20]:
df_tot.tail()

,File_Name,Labels,Embeds
2822,file_2474.txt,-1,"[[-0.1262939, -0.019628199, -0.005916388, -0.16452256, 0.09194111, -0.08190768, 0.009038988, 0.1..."
2823,file_863.txt,-1,"[[-0.16330343, -0.0963432, 0.08022924, 0.01302716, 0.13003008, -0.18599892, 0.19946839, 0.130659..."
2824,file_1547.txt,-1,"[[0.0355055, -0.112331845, 0.09806109, -0.035105072, -0.005587622, -0.42070362, 0.14120506, 0.29..."
2825,file_1292.txt,-1,"[[0.11228424, -0.20076993, -0.0851114, -0.1541715, -0.0011653934, -0.10996413, 0.182178, 0.09550..."
2826,file_1910.txt,-1,"[[-0.0010850747, -0.053630237, 0.17418769, -0.12784621, 0.1681908, -0.33785436, 0.052511718, 0.1..."


In [21]:
df_raw = df_tot[df_tot.Labels != -1]
df_test = df_tot[df_tot.Labels == -1]
df_test.reset_index(inplace= True,drop=True)
len(df_raw),len(df_test)

(1978, 849)

In [22]:
df_raw.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
10      2
18      1
Name: Labels, dtype: int64

In [23]:
#For 5 fold stratified cross validation, the value counts for each label must be atleast 5
df_rare = df_raw[df_raw.Labels.apply(lambda x: x in [12,9,3,17,10,18])]
df_new = pd.concat([df_raw] + [df_rare]*4)
df_new.reset_index(inplace= True,drop=True)

df_new.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
12     20
20     18
13     15
9      15
3      10
10     10
17     10
21      9
18      5
Name: Labels, dtype: int64

In [0]:
#Creating a new column and assigning each row into a fold
df_new["kfold"] = -1
skf = StratifiedKFold()
for fold, (train_idx, val_idx) in enumerate(skf.split(X=df_new, y=df_new['Labels'].values)):
                    df_new.loc[val_idx, 'kfold'] = fold

In [0]:
#Create a new dataframe with array values
def create_df_embeds(feat_array, df):
    cols_num = feat_array[0].shape[0]
    col_names = ['feat_' + str(i+1) for i in range(cols_num)]
    df_feat = pd.DataFrame(data = feat_array,columns = col_names)
    df_fit = pd.concat([df, df_feat], axis=1)
    return df_fit,col_names

In [0]:
def train(df_fit,classifier,feat_cols,label_col = 'Labels'):
    
    folds = df_fit.kfold.nunique()
    FOLD_MAPPPING = {fold:list(set(range(folds)) - set([fold]))  for fold in range(folds)}
    clfs = [classifier]*folds
    
    train_losses = []
    train_accuracies = []
    valid_losses = []
    valid_accuracies = []
    
    for FOLD in range(5):
        
        train_df = df_fit[df_fit.kfold.isin(FOLD_MAPPPING.get(FOLD))].reset_index(drop=True)
        valid_df = df_fit[df_fit.kfold==FOLD].reset_index(drop=True)
        
        clf = clfs[FOLD]
        clf.fit(train_df[feat_cols].values, train_df[label_col])
        
        train_probs = clf.predict_proba(train_df[feat_cols].values)
        train_preds = clf.predict(train_df[feat_cols].values)
        
        train_accuracy = accuracy_score(train_df[label_col], train_preds)
        train_loss = log_loss(train_df[label_col], train_probs)
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        valid_probs = clf.predict_proba(valid_df[feat_cols].values)
        valid_preds = clf.predict(valid_df[feat_cols].values)
        
        valid_accuracy = accuracy_score(valid_df[label_col], valid_preds)
        valid_loss = log_loss(valid_df[label_col], valid_probs)
        
        valid_losses.append(valid_loss)
        valid_accuracies.append(valid_accuracy)
        
    mean_train_loss,std_train_loss = np.mean(train_losses),np.std(train_losses)
    mean_train_accuracy,std_train_accuracy  = np.mean(train_accuracies),np.std(train_accuracies)

    mean_valid_loss,std_valid_loss = np.mean(valid_losses),np.std(valid_losses)
    mean_valid_accuracy,std_valid_accuracy  = np.mean(valid_accuracies),np.std(valid_accuracies)

    print(f'train loss = {mean_train_loss} with {np.round((std_train_loss/mean_train_loss)*100, 2)}% error')
    print(f'valid loss = {mean_valid_loss} with {np.round((std_valid_loss/mean_valid_loss)*100, 2)}% error')

    print(f'train accuracy = {mean_train_accuracy} with {np.round((std_train_accuracy/mean_train_accuracy)*100, 2)}% error')
    print(f'valid accuracy = {mean_valid_accuracy} with {np.round((std_valid_accuracy/mean_valid_accuracy)*100, 2)}% error')
    
    return clfs

In [0]:
def predict(df,clfs,feat_cols,identifier = ['File_Name']):
    df_fit = df[feat_cols]
    num_targets = len(clfs[0].classes_)
    for i,clf in enumerate(clfs):
        probs = clf.predict_proba(df_fit.values)
        if i == 0:
            predictions = probs
        else:
            predictions += probs
    
    predictions /= len(clfs)
    
    pred_df = pd.DataFrame(predictions,columns=list(range(num_targets)))
    df_subm = pd.concat([df[identifier], pred_df], axis=1)
       
    return df_subm

In [28]:
mean_array = np.stack((df_new.Embeds.apply(lambda x: x.mean(axis = 0)).values),axis = 0)
df_mean,cols = create_df_embeds(mean_array, df_new)
classif = LogisticRegression(random_state=seed)
LR_clfs = train(df_mean,classif,cols)

train loss = 1.6917428895320423 with 1.45% error
valid loss = 2.0303416478260976 with 1.25% error
train accuracy = 0.3731562288108078 with 2.79% error
valid accuracy = 0.2630336113094734 with 5.09% error


In [29]:
test_mean_array = np.stack((df_test.Embeds.apply(lambda x: x.mean(axis = 0)).values),axis = 0)
df_test_mean,cols = create_df_embeds(test_mean_array, df_test)
df_submission = predict(df_test_mean,LR_clfs,cols)
df_submission.head()

,File_Name,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,file_2300.txt,0.001829,0.008076,0.002059,0.004352,0.146735,0.013130,0.286642,0.007446,0.010214,0.004847,0.000060,0.194803,0.000360,0.001027,0.020801,0.058224,0.017925,0.002229,0.000405,0.217374,0.000220,0.001242
1,file_809.txt,0.002700,0.004175,0.000680,0.001996,0.710791,0.017324,0.073842,0.001144,0.006624,0.002924,0.000016,0.000303,0.000582,0.000702,0.000973,0.155835,0.002312,0.000553,0.000377,0.015853,0.000085,0.000209
2,file_1383.txt,0.095004,0.333355,0.003540,0.000359,0.004653,0.000106,0.053628,0.003291,0.019389,0.000069,0.000032,0.005490,0.000012,0.000109,0.001753,0.000659,0.368461,0.002111,0.001176,0.098888,0.007732,0.000181
3,file_983.txt,0.011831,0.022818,0.001852,0.003467,0.163525,0.149875,0.274547,0.001712,0.010987,0.010577,0.000202,0.000755,0.001996,0.002593,0.012378,0.254647,0.009736,0.002047,0.000701,0.062396,0.000768,0.000590
4,file_1713.txt,0.198234,0.257614,0.043991,0.008591,0.026813,0.012197,0.080509,0.037501,0.064655,0.001037,0.000221,0.079341,0.000895,0.021821,0.044294,0.019674,0.046538,0.002853,0.002131,0.038443,0.007280,0.005366


In [0]:
def create_file(df,file_name):
    file_path = os.path.join(cur_dir,(file_name + '.xlsx'))
    df.to_excel(file_path, index = False)

In [0]:
create_file(df_submission,'bert_mean')